In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/retail-data2/orders_data")
dbutils.fs.mkdirs("dbfs:/FileStore/retail-data2/checkpoint01")


Out[1]: True

In [0]:
landing_path = "dbfs:/FileStore/retail-data2"
orders_data = landing_path + "/orders_data"
checkpointpath1 = landing_path + "/checkpoint01"

In [0]:
dbutils.fs.ls("dbfs:/FileStore/retail-data2/checkpoint01/")


Out[3]: [FileInfo(path='dbfs:/FileStore/retail-data2/checkpoint01/_schemas/', name='_schemas/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/retail-data2/checkpoint01/commits/', name='commits/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/retail-data2/checkpoint01/metadata', name='metadata', size=45, modificationTime=1752209578000),
 FileInfo(path='dbfs:/FileStore/retail-data2/checkpoint01/offsets/', name='offsets/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/retail-data2/checkpoint01/sources/', name='sources/', size=0, modificationTime=0)]

In [0]:
from pyspark.sql.functions import input_file_name, current_timestamp


In [0]:
# STREAMING READ FROM FILES USING AUTO LOADER
orders_df = spark.readStream \
    .format("cloudFiles") \
        .option("cloudFiles.format", "csv") \
            .option("cloudFiles.inferSchema", "true") \
                .option("cloudFiles.inferColumnTypes", "true") \
                    .option("cloudFiles.schemaLocation", checkpointpath1) \
                        .option("cloudFiles.schemaEvolutionMode", "none") \
                            .option("header", "true") \
                                .load(orders_data) \
                                    .withColumn("file_name", input_file_name()) \
                                        .withColumn("time_of_load", current_timestamp())

In [0]:
orders_df.writeStream \
    .format("delta") \
        .option("checkpointLocation", checkpointpath1) \
            .queryName("orders_streaming_query") \
                .outputMode("append") \
                    .toTable("ordersdeltanew2")


Out[7]: <pyspark.sql.streaming.query.StreamingQuery at 0x7fe25daa1460>

In [0]:
dbutils.fs.rm("dbfs:/mnt/your-path-to-delta-table", recurse=True)


In [0]:
%sql
drop table default.ordersdeltanew2

org.apache.spark.sql.catalyst.analysis.NoSuchTableException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `spark_catalog`.`default`.`ordersdeltanew2` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01
	at org.apache.spark.sql.errors.QueryCompilationErrors$.noSuchTableError(QueryCompilationErrors.scala:1673)
	at org.apache.spark.sql.execution.datasources.v2.DropTableExec.run(DropTableExec.scala:42)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.$anonfun$result$2(V2CommandExec.scala:48)
	at org.apache.spark.sql.execution.SparkPlan.runCommandWithAetherOff(SparkPlan.scala:178)
	at org.apache.spark.sql.execution.SparkPlan.runCommandInAetherOrSpark(SparkPlan.scala:189)
	at org.apache.spark.sql.execution.datasources.v

In [0]:
%sql
select * from ordersdeltanew2


OrderID,OrderDate,CustomerID,TotalAmount,Status,file_name,time_of_load
1,2023-01-01,25,6202.83,Cancelled,dbfs:/FileStore/retail-data2/orders_data/orders1.csv,2025-07-11T04:56:04.367+0000
2,2023-01-02,32,9871.87,Cancelled,dbfs:/FileStore/retail-data2/orders_data/orders1.csv,2025-07-11T04:56:04.367+0000
3,2023-01-03,78,6814.71,Completed,dbfs:/FileStore/retail-data2/orders_data/orders1.csv,2025-07-11T04:56:04.367+0000
4,2023-01-04,89,4972.72,Pending,dbfs:/FileStore/retail-data2/orders_data/orders1.csv,2025-07-11T04:56:04.367+0000
5,2023-01-05,91,5150.9,Pending,dbfs:/FileStore/retail-data2/orders_data/orders1.csv,2025-07-11T04:56:04.367+0000
6,2023-01-06,6,6603.76,Pending,dbfs:/FileStore/retail-data2/orders_data/orders1.csv,2025-07-11T04:56:04.367+0000
7,2023-01-07,20,7080.93,Cancelled,dbfs:/FileStore/retail-data2/orders_data/orders1.csv,2025-07-11T04:56:04.367+0000
8,2023-01-08,15,4101.39,Cancelled,dbfs:/FileStore/retail-data2/orders_data/orders1.csv,2025-07-11T04:56:04.367+0000
9,2023-01-09,16,1755.19,Pending,dbfs:/FileStore/retail-data2/orders_data/orders1.csv,2025-07-11T04:56:04.367+0000
10,2023-01-10,34,4953.16,Pending,dbfs:/FileStore/retail-data2/orders_data/orders1.csv,2025-07-11T04:56:04.367+0000
